

:warning: Due to the changes in PubMed API, this notebook does not work as expected anymore :warning:
# Search PubMed and store abstracts

This notebook searches PubMed for all the defined queries in the EFO ontology (i.e., properties of type `efo:pubmedQuery`), then cleanups and aggregates the XML results, and stores all the search hits in a single CSV.

**NOTE**: Notebooks should be executed from the project root folder. All paths are relative to the project root.


## Input

- `data/ontologies/efo.owl`: EFO ontology.

## Output

- `data/pubmed/abstracts_2023.csv.gz`: Aggregated abstracts are stored as a single compressed file, containing the following columns:
    - `category` (`str`): either `CognitiveTask` or `CognitiveConstruct`
    - `subcategory` (`str`): task or construct name
    - `pmid` (`long`): PubMed identifier
    - `doi` (`str`): DOI
    - `year` (`int`): publication year in `yyyy` format
    - `title` (`str`): publication title
    - `abstract` (`str`): publication abstract
    - `journal_title` (`str`): full journal title
    - `journal_iso_abbreviation` (`str`): Abbreviated journal title
    - `mesh` (`str`, deprecated): A list of Medical Subject Headings which indicates the field of research and other topics. We only keep major topics.

## Requirements

First set the `NCBI_API_KEY` environment variable ([How do I obtain an API Key through an NCBI account
](https://support.nlm.nih.gov/knowledgebase/article/KA-05317/en-us)). Then activate the `cogtext` conda environment:

```bash
mamba activate cogtext
```


In [2]:
%reload_ext autoreload
%autoreload 3

from owlready2 import get_ontology
from pathlib import Path
from dotenv import load_dotenv; load_dotenv()  # to load NCBI_API_KEY env variable
import pandas as pd
import re
from tqdm import tqdm
from IPython.display import clear_output

from src.cogtext.datasets.pubmed import convert_xml_to_csv


In [3]:
DEBUG = True

# collect data for the following categories
CATEGORIES = ['CognitiveTask', 'CognitiveConstruct']

OUTPUT_PATH = 'data/pubmed/abstracts_2023.csv.gz'

OWL_FILE = 'data/ontologies/efo.owl'
ONTOLOGY = get_ontology(OWL_FILE).load()

In [4]:
for category in CATEGORIES:
  pubmed_queries = {e.name:e.pubmedQuery[0] for e in ONTOLOGY[category].descendants() if len(e.pubmedQuery) > 0}
  print(f'EF ontology contains {len(pubmed_queries)} PubMed queries for {category}s.')

EF ontology contains 126 PubMed queries for CognitiveTasks.
EF ontology contains 72 PubMed queries for CognitiveConstructs.


In [8]:
from joblib import Parallel, delayed

import os
import subprocess
import shlex
import xml
import xml.etree.ElementTree as ET


def pubmed_pipeline(subcategory, query, csv_dir, overwrite=False,
                    edirect_dir='/home/morteza/edirect'):
    subcategory = subcategory.replace('/', '')
    fname = Path('data/pubmed/.cache') / (subcategory + '.asn')

    if overwrite or not fname.exists():
        edirect_env = os.environ.copy()
        edirect_env['PATH'] = f"{edirect_dir}:{edirect_env['PATH']}"

        query = query.replace('"', '\\"')
        esearch_cmd = f'esearch -db pubmed -query "{query}"'
        efetch_cmd = f'efetch -format asn.1'

        esearch = subprocess.Popen(shlex.split(esearch_cmd),
                                    stdout=subprocess.PIPE, env=edirect_env)
        out, err = esearch.communicate()

        efetch = subprocess.Popen(shlex.split(efetch_cmd), stdin=subprocess.PIPE,
                                    stdout=subprocess.PIPE, env=edirect_env)
        out, err = efetch.communicate(input=out)

        fname.parent.mkdir(parents=True, exist_ok=True)

        with open(fname, 'wb') as f:
            f.write(out)

        print(f'[EDirect] Finished {query}')


    convert_xml_to_csv(subcategory, csv_dir=csv_dir, overwrite_existing=overwrite)

for category in CATEGORIES:

    # init folder
    csv_dir = Path(f'data/pubmed/{category}')
    csv_dir.mkdir(parents=True, exist_ok=True)

    queries = {subc.name: subc.pubmedQuery[0]
               for subc in ONTOLOGY[category].descendants()
               if len(subc.pubmedQuery) > 0}

    Parallel(n_jobs=5)(delayed(
        pubmed_pipeline
    )(subcategory, query, csv_dir=csv_dir, overwrite=True)
                       for subcategory, query in queries.items())

# clear_output()
print('Done!')

[EDirect] Finished (\"Backwards Color Recall task\"[TIAB])
[EDirect] Finished (\"Toy Sort\"[TIAB])
[EDirect] Finished (Degraded Vigilance task* \"PVT\"[TIAB])
[EDirect] Finished (\"Balance Beam task\"[TIAB]) OR (\"Balance Beam test\"[TIAB])
[EDirect] Finished (\"Lexical Fluency\"[TIAB])
[EDirect] Finished (\"Controlled Attention task\"[TIAB])
[EDirect] Finished (\"snack delay\"[TIAB]) AND (task* OR test* OR paradigm*[TIAB])
[EDirect] Finished \"Stimulus Selective Stop Signal Task\"[TIAB]
[EDirect] Finished (\"Counting Span\"[TIAB])
[EDirect] Finished (\"Self Control Schedule\"[TIAB])
[EDirect] Finished (\"Pencil Tapping\"[TIAB]) OR (\"PEG task\"[TIAB]) OR (\"PEG test\"[TIAB])


/home/morteza/edirect/efetch: line 1116: 695991 Done                    echo "$res"
     695992                       | join-into-groups-of "$chunk"
     695994 Killed                  | while read uids; do
    RunWithFetchArgs nquire -url "$base" efetch.fcgi -db "$dbase" -id "$uids" -rettype "$format" -retmode "$mode";
done
/home/morteza/edirect/efetch: line 1116: 695130 Exit 137                if [ "$needHistory" = false ]; then
    res=$(LimitUidList "$dbase"); echo "$res" | join-into-groups-of "$chunk" | while read uids; do
        RunWithFetchArgs nquire -url "$base" efetch.fcgi -db "$dbase" -id "$uids" -rettype "$format" -retmode "$mode";
    done;
else
    GenerateHistoryChunks "$chunk" "$min" "$max" | while read fr chnk; do
        RunWithFetchArgs nquire -url "$base" efetch.fcgi -query_key "$qry_key" -WebEnv "$web_env" -retstart "$fr" -retmax "$chnk" -db "$dbase" -rettype "$format" -retmode "$mode";
    done;
fi
     695131 Killed                  | if [ "$format" = "json" ] |

KeyboardInterrupt: 

Now that we have separate CSV files, we combine them and store the whole PubMed abstracts corpus as a single compressed CSV file:

In [ ]:
# aggregation

corpus_files = Path('data/pubmed/').glob('**/*.csv')

dfs = []

for fname in tqdm(corpus_files):

  # find categories from the file name
  cats = re.findall('.*/pubmed/(.*)/(.*)\\.csv', str(fname))

  # ignore other csv files
  if len(cats) == 0:
    continue

  category = cats[0][0]
  subcategory = cats[0][1]

  df = pd.read_csv(fname)
  df['category'] = category
  df['subcategory'] = subcategory
  dfs.append(df)

# now aggregate all the data and store the compressed csv output (takes ~ 2min).
pd.concat(dfs).to_csv(OUTPUT_PATH, index=False, compression='gzip')

In [ ]:
pm2021 = pd.read_csv('data/pubmed/abstracts_2021.csv.gz', compression='gzip')

In [ ]:
pm2021.groupby('subcategory').size().sort_values(ascending=False).head(10)

In [ ]:
pm2023 = pd.read_csv('data/pubmed/abstracts_2023.csv.gz', compression='gzip')